# Database Design for a bank

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

I have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like me to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied me with a csv file called `"bank_marketing.csv"`, which I will need to clean, reformat, and split, in order to save separate files based on the tables I will create. It is recommended to use `pandas` for these tasks.

Lastly, I will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, I'll save SQL files as multiline string variables that they can then use to create the database on their end. 

I have been asked to design a database that will have three tables:

## client

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `id` | `serial` | Client ID - primary key | `client_id` |
| `age` | `integer` | Client's age in years | `age` |
| `job` | `text` | Client's type of job | `job` |
| `marital` | `text` | Client's marital status | `marital` | 
| `education` | `text` | Client's level of education | `education` |
| `credit_default` | `boolean` | Whether the client's credit is in default | `credit_default` |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) | `housing` | 
| `loan` | `boolean` | Whether the client has an existing personal loan | `loan` |

<br>

## campaign

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `campaign_id` | `serial` | Campaign ID - primary key | N/A - new column |
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | `campaign` |
| `contact_duration` | `integer` | Last contact duration in seconds | `duration` |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) | `pdays` |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | `previous` |
| `previous_outcome` | `boolean` | Outcome of the previous campaign | `poutcome` |
| `campaign_outcome` | `boolean` | Outcome of the current campaign | `y` |
| `last_contact_date` | `date` | Last date the client was contacted | A combination of `day`, `month`, and the newly created `year` |

<br>

## economics

| column | data type | description | original column in dataset |
|--------|-----------|-------------|----------------------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table | `client_id` |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) | `emp_var_rate` |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | `cons_price_idx` |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) | `euribor3m` |
| `number_employed` | `float` | Number of employees (quarterly indicator)| `nr_employed` |

In [5]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# read in the csv file
df = pd.read_csv('bank_marketing.csv')


# The data is split into three dataframes; client, campaign, economics 
client = df[["client_id", "age", "job", "marital", "education", "credit_default", "housing", "loan"]]

campaign = df[["client_id", "campaign", "month", "day", "duration", "pdays", "previous", "poutcome", "y"]]

economics = df[["client_id", "emp_var_rate", "cons_price_idx", "euribor3m", "nr_employed"]]


# Renaming columns
client = client.rename(columns={"client_id":"id"})

campaign = campaign.rename(columns={"duration":"contact_duration", "previous":"previous_campaign_contacts", "y":"campaign_outcome", "poutcome":"previous_outcome", "campaign":"number_contacts"})

economics = economics.rename(columns={"euribor3m":"euribor_three_months", "nr_employed":"number_employed"})


# Cleaning columns
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)
client["job"] = client["job"].str.replace(".", "")
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({1:True, 0:False})
campaign["previous_outcome"] = campaign["previous_outcome"].replace("nonexistent", np.NaN)
campaign["previous_outcome"] = campaign["previous_outcome"].map({1:True, 0:False})


# Creating a datetime column in "year-month-day" column
df["last_contact_date"] = "1"
df["month"] = campaign["month"].str.capitalize()
df["year"] = "2022"
df["day"] = campaign["day"].astype(str)

# Adding the columns together
df["last_contact_date"] = df["month"] + "-" + df["year"] + "-" + df["day"]

# Converting last_contact_date to datetime
df["last_contact_date"] = pd.to_datetime(df["last_contact_date"], dayfirst=True)

# Deleting Columns
df.drop(["month", "day", "year"], axis=1, inplace=True)

# Saving the three DataFrames to csv files
client.to_csv(index=False)
campaign.to_csv(index=False)
economics.to_csv(index=False)



# Designing the database; using SQL code as a multi-line string to create tables
client_table = """
CREATE TABLE client
(id SERIAL PRIMARY KEY,
age INTEGER,
job TEXT,
marital TEXT,
education TEXT,
credit_default BOOLEAN,
housing BOOLEAN,
loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""
campaign_table = """
CREATE TABLE campaign
(campaign_id SERIAL PRIMARY KEY,
client_id SERIAL references client (id),
number_contacts INTEGER,
contact_duration INTEGER,
pdays INTEGER,
previous_campaign_contacts INTEGER,
previous_outcome BOOLEAN,
campaign_outcome BOOLEAN,
last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER
"""
economics_table = """
CREATE TABLE economics
(client_id SERIAL references client (id), 
emp_var_rate FLOAT,
cons_price_idx FLOAT,
euribor_three_months FLOAT,
number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""

In [9]:
client

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic_4y,no,no,no
1,1,57,services,married,high_school,unknown,no,no
2,2,37,services,married,high_school,no,yes,no
3,3,40,admin,married,basic_6y,no,no,no
4,4,56,services,married,high_school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional_course,no,yes,no
41184,41184,46,blue-collar,married,professional_course,no,no,no
41185,41185,56,retired,married,university_degree,no,yes,no
41186,41186,44,technician,married,professional_course,no,no,no


In [10]:
campaign

,client_id,number_contacts,month,day,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,may,13,261,999,0,NaN,NaN
1,1,1,may,19,149,999,0,NaN,NaN
2,2,1,may,23,226,999,0,NaN,NaN
3,3,1,may,27,151,999,0,NaN,NaN
4,4,1,may,3,307,999,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
41183,41183,1,nov,30,334,999,0,NaN,NaN
41184,41184,1,nov,6,383,999,0,NaN,NaN
41185,41185,2,nov,24,189,999,0,NaN,NaN
41186,41186,1,nov,17,442,999,0,NaN,NaN


In [11]:
economics

,client_id,emp_var_rate,cons_price_idx,euribor_three_months,number_employed
0,0,1.1,93.994,4.857,5191.0
1,1,1.1,93.994,4.857,5191.0
2,2,1.1,93.994,4.857,5191.0
3,3,1.1,93.994,4.857,5191.0
4,4,1.1,93.994,4.857,5191.0
...,...,...,...,...,...
41183,41183,-1.1,94.767,1.028,4963.6
41184,41184,-1.1,94.767,1.028,4963.6
41185,41185,-1.1,94.767,1.028,4963.6
41186,41186,-1.1,94.767,1.028,4963.6


In [12]:
# The database was created successfully, with the three tables (client,campaign and economics) having outputs just as was instructed.